In [1]:
import pandas as pd
import time
from FlightRadar24.errors import AirportNotFoundError  # Import the AirportNotFoundError
import copy
import json
from datetime import datetime


In [2]:
from FlightRadar24 import FlightRadar24API
fr_api = FlightRadar24API()

In [3]:
flights = fr_api.get_flights()
airports = fr_api.get_airports()

In [4]:
df_airports = pd.DataFrame(airports)
df_airports.to_csv("api_data/airports")

In [5]:
df_flights = pd.DataFrame(flights)
df_flights.to_csv("api_data/flights")

# Airports details

Récupère dans une liste les détails pour chaque aéroports qui se trouvent dans la liste **airports**

In [6]:
airports = pd.read_csv("api_data/airports",sep = ",")

In [151]:
airports_details3 = []
notfound = []

In [ ]:
for c in airports_bis:  
    try:
        airports_details3.append(fr_api.get_airport_details(c.split()[0][2:-1]))
        time.sleep(5)  # Wait for 5 second between API calls
    except AirportNotFoundError as e:
        notfound.append(c)

In [ ]:
airports_d = [airports_details3[i]["airport"]["pluginData"]["details"]["code"]["icao"] for i in range(len(airports_details3))]

In [ ]:
airports_bis = [icao for icao in airports['0'] if icao.split()[0][2:-1] not in airports_d]

In [ ]:
output_file = open('airports_details_final', 'w', encoding='utf-8')
for dic in airports_details3:
    json.dump(dic, output_file) 
    output_file.write("\n")

## Création du dataset final

In [6]:
airport_list = []
with open(r'airports_details_final', 'r', encoding='utf-8') as input_file:
    for line_number, line in enumerate(input_file, start=1):
        try:
            airport_list.append(json.loads(line))
        except json.JSONDecodeError as e:
            print(f"Erreur de décodage JSON à la ligne {line_number}: {e}")

Erreur de décodage JSON à la ligne 5454: Expecting ',' delimiter: line 1 column 1462 (char 1461)


***
En exécutant la fonction **fr_api.get_airport_details(code_airport)**, il n'a rien trouvé (code introuvable) pour certain aéroports qui se trouvent dans la liste obtenu avec **fr_api.get_airports()**

In [7]:
len(airport_list)

5453

In [8]:
len(airports)

5498

### Se familiariser avec le fichier json

In [9]:
for i in range(len(airport_list)):
    airport_list[i]['airport']['pluginData']['details'].pop('airportImages')
    airport_list[i].pop('aircraftImages')

In [90]:
print(airport_list[0]['airport']['pluginData'].keys())
print(airport_list[0]['airport']['pluginData']['details'].keys())
print(airport_list[0]['airport']['pluginData']['flightdiary'].keys())
print(airport_list[0]['airport']['pluginData']['schedule'].keys())
print(airport_list[0]['airport']['pluginData']['weather'].keys())
print(airport_list[0]['airport']['pluginData']['aircraftCount'].keys())
# print(airports_details[0]['airport']['pluginData']['runways'].keys())

dict_keys(['details', 'flightdiary', 'schedule', 'weather', 'aircraftCount', 'runways'])
dict_keys(['name', 'code', 'delayIndex', 'stats', 'position', 'timezone', 'url', 'visible'])
dict_keys(['url', 'ratings', 'comment', 'reviews', 'evaluation'])
dict_keys(['arrivals', 'departures', 'ground'])
dict_keys(['metar', 'time', 'qnh', 'dewpoint', 'humidity', 'pressure', 'sky', 'flight', 'wind', 'temp', 'elevation', 'cached'])
dict_keys(['ground', 'onGround'])


In [143]:
airport_list[704]['airport']['pluginData']['details']['code']['icao']

'YBUN'

In [154]:
#Schedule qui m'intéresse: --> arrivals/departures
print(airport_list[704]['airport']['pluginData']['schedule']['departures'].keys())
print(airport_list[704]['airport']['pluginData']['schedule']['departures']['data'])
print()
print(airport_list[704]['airport']['pluginData']['schedule']['arrivals']['data'])


dict_keys(['item', 'page', 'timestamp', 'data'])
[{'flight': {'identification': {'id': None, 'row': 5484743428, 'number': {'default': None, 'alternative': None}, 'callsign': 'VHYUO', 'codeshare': None}, 'status': {'live': False, 'text': 'Estimated dep 06:00', 'icon': 'green', 'estimated': None, 'ambiguous': False, 'generic': {'status': {'text': 'estimated', 'type': 'departure', 'color': 'green', 'diverted': None}, 'eventTime': {'utc': 1710453600, 'local': 1710482400}}}, 'aircraft': {'model': {'code': 'C172', 'text': 'Cessna 172R Skyhawk'}, 'registration': 'VH-YUO', 'country': {'id': 14, 'name': 'Australia', 'alpha2': 'AU', 'alpha3': 'AUS'}, 'hex': '7C7C5E', 'restricted': False, 'serialNo': None, 'age': {'availability': True}, 'availability': {'serialNo': True, 'age': True}}, 'owner': None, 'airline': None, 'airport': {'origin': {'timezone': {'name': 'Australia/Perth', 'offset': 28800, 'abbr': 'AWST', 'abbrName': 'Australian Western Standard Time', 'isDst': False}, 'info': {'terminal': 

#### Schedule

In [147]:
#Schedule qui m'intéresse: --> arrivals/departures
print(airport_list[0]['airport']['pluginData']['schedule']['departures'].keys())
for i in range(len(airport_list[0]['airport']['pluginData']['schedule']['departures']['data'])):
    for key in airport_list[0]['airport']['pluginData']['schedule']['departures']['data'][3]['flight'].keys():
        print(key)
        print(airport_list[0]['airport']['pluginData']['schedule']['departures']['data'][3]['flight'][key])
    break

# for arrival in airport_list[0]['airport']['pluginData']['schedule']['arrivals']['data']:
#     print(arrival['flight'])

dict_keys(['item', 'page', 'timestamp', 'data'])
identification
{'id': None, 'row': 5481563816, 'number': {'default': 'IB511', 'alternative': None}, 'callsign': None, 'codeshare': ['AA8546', 'AV6197', 'AY5606', 'BA7111', 'CX1862', 'LA7030', 'VY5034', 'JL9414']}
status
{'live': False, 'text': 'Estimated dep 06:30', 'icon': 'green', 'estimated': None, 'ambiguous': False, 'generic': {'status': {'text': 'estimated', 'type': 'departure', 'color': 'green', 'diverted': None}, 'eventTime': {'utc': 1710480600, 'local': 1710484200}}}
aircraft
{'model': {'code': '319', 'text': ''}, 'hex': '', 'registration': '', 'serialNo': '', 'images': None}
owner
None
airline
{'name': 'Iberia', 'code': {'iata': 'IB', 'icao': 'IBE'}, 'short': 'Iberia'}
airport
{'origin': {'timezone': {'name': 'Europe/Madrid', 'offset': 3600, 'abbr': 'CET', 'abbrName': 'Central European Time', 'isDst': False}, 'info': {'terminal': 'T1', 'baggage': None, 'gate': '3'}}, 'destination': {'code': {'iata': 'MAD', 'icao': 'LEMD'}, 'tim

#### Weather

In [92]:
print(airport_list[0]['airport']['pluginData']['weather'].items())

dict_items([('metar', 'LECO 141900Z 22004KT 180V260 9999 FEW030 15/12 Q1015 NOSIG'), ('time', 1710442800), ('qnh', 1015), ('dewpoint', {'celsius': 12, 'fahrenheit': 54}), ('humidity', 82), ('pressure', {'hg': 30, 'hpa': 1015}), ('sky', {'condition': {'text': 'Cloudy'}, 'visibility': {'km': 99999000, 'mi': 62, 'nmi': 54}}), ('flight', {'category': None}), ('wind', {'direction': {'degree': 220, 'text': 'From southwest'}, 'speed': {'kmh': 7, 'kts': 4, 'mph': 4, 'text': 'Calm'}}), ('temp', {'celsius': 15, 'fahrenheit': 59}), ('elevation', {'m': 97, 'ft': 318}), ('cached', 1710444827)])


In [93]:
sky_condition = airport_list[0]['airport']['pluginData']['weather']["sky"]["condition"]["text"]
sky_visibility = str("{:.2f}".format(int(airport_list[0]['airport']['pluginData']['weather']["sky"]["visibility"]["nmi"])*1.852))
wind_direction_degree = airport_list[0]['airport']['pluginData']['weather']["wind"]["direction"]["degree"]
wind_direction_text = airport_list[0]['airport']['pluginData']['weather']["wind"]["direction"]["text"]
wind_speed_desc = airport_list[0]['airport']['pluginData']['weather']["wind"]["speed"]["text"]
wind_speed_km = airport_list[0]['airport']['pluginData']['weather']["wind"]["speed"]["kmh"]
wind_speed_kts = airport_list[0]['airport']['pluginData']['weather']["wind"]["speed"]["kts"]
temperature = airport_list[0]['airport']['pluginData']['weather']["temp"]["celsius"]

print(sky_condition)
print(sky_visibility)
print(wind_direction_degree,wind_direction_text)
print(wind_speed_desc)
print(wind_speed_km)
print(wind_speed_kts)
print(temperature)

Cloudy
100.01
220 From southwest
Calm
7
4
15


### Récupération des informations pertinentes

In [112]:
country_list = []
all_airports = []
arr_flight_identification = []
arr_flight_status = []
arr_flight_aircraft_code = []
arr_flight_aircraft_name = []
arr_flight_aircraft_registration = []
arr_flight_aircraft_owner_country = []
arr_flight_aircraft_owner_airline_name = []
arr_flight_aircraft_owner_airline_code = []
dep_flight_identification = []
dep_flight_status = []
dep_flight_aircraft_code = []
dep_flight_aircraft_name = []
dep_flight_aircraft_registration = []
dep_flight_aircraft_owner_country = []
dep_flight_aircraft_owner_airline_name = []
dep_flight_aircraft_owner_airline_code = []
arr_came_from_country = []
arr_came_from_airport_name = []
arr_came_from_airport_code = []
arr_scheduled_departure = []
arr_scheduled_arrival = []
arr_real_departure = []
arr_real_arrival = []
arr_estimated_departure = []
arr_estimated_arrival = []
arr_duree = []
dep_country_destination = []
dep_airport_name_destination = []
dep_airport_code_icao_destination = []
dep_scheduled_departure = []
dep_scheduled_arrival = []
dep_real_departure = []
dep_real_arrival = []
dep_estimated_departure = []
dep_estimated_arrival = []
dep_duree = []
sky_condition = []
sky_visibility = []
wind_direction = []
wind_speed_desc = []
wind_speed_km = []
wind_speed_kts = []
temperature = []

for i in range(len(airport_list)):
    occurences = 0
    for arrival,departure in zip(airport_list[i]['airport']['pluginData']['schedule']['arrivals']['data'],airport_list[i]['airport']['pluginData']['schedule']['departures']['data']):
        occurences+=1
        country_list.append(airport_list[i]['airport']['pluginData']['details']['position']['country']['name'])
        all_airports.append(airport_list[i]['airport']['pluginData']['details']['name'])
        try:
            arr_flight_identification.append(arrival['flight']['identification']['number']['default'])
        except (KeyError, TypeError):
            arr_flight_identification.append(None)
        try:
            arr_flight_status.append(arrival['flight']['status']['text'])
        except (KeyError, TypeError):
            arr_flight_status.append(None)
        try:
            arr_flight_aircraft_code.append(arrival['flight']['aircraft']['model']['code'])
        except (KeyError, TypeError):
            arr_flight_aircraft_code.append(None)
        try:
            arr_flight_aircraft_name.append(arrival['flight']['aircraft']['model']['text'])
        except (KeyError, TypeError):
            arr_flight_aircraft_registration.append(None)
        try:
            arr_flight_aircraft_registration.append(arrival['flight']['aircraft']['registration'])
        except (KeyError, TypeError):
            arr_flight_aircraft_registration.append(None)
        try:
            arr_flight_aircraft_owner_country.append(arrival['flight']['aircraft']['country']['name'])
        except (KeyError, TypeError):
            arr_flight_aircraft_owner_country.append(None)
        try:
            arr_flight_aircraft_owner_airline_name.append(arrival['flight']['owner']['name'])
        except (KeyError, TypeError):
            arr_flight_aircraft_owner_airline_name.append(None)
        try:
            arr_flight_aircraft_owner_airline_code.append(arrival['flight']['owner']['code']['icao'])
        except (KeyError, TypeError):
            arr_flight_aircraft_owner_airline_code.append(None)


        try:
            dep_flight_identification.append(departure['flight']['identification']['number']['default'])
        except (KeyError, TypeError):
            dep_flight_identification.append(None)
        try:
            dep_flight_status.append(departure['flight']['status']['text'])
        except (KeyError, TypeError):
            arr_flight_status.append(None)
        try:
            dep_flight_aircraft_code.append(departure['flight']['aircraft']['model']['code'])
        except (KeyError, TypeError):
            dep_flight_aircraft_code.append(None)
        try:
            dep_flight_aircraft_name.append(departure['flight']['aircraft']['model']['text'])
        except (KeyError, TypeError):
            dep_flight_aircraft_registration.append(None)
        try:
            dep_flight_aircraft_registration.append(departure['flight']['aircraft']['registration'])
        except (KeyError, TypeError):
            dep_flight_aircraft_registration.append(None)
        try:
            dep_flight_aircraft_owner_country.append(departure['flight']['aircraft']['country']['name'])
        except (KeyError, TypeError):
            dep_flight_aircraft_owner_country.append(None)
        try:
            dep_flight_aircraft_owner_airline_name.append(departure['flight']['owner']['name'])
        except (KeyError, TypeError):
            dep_flight_aircraft_owner_airline_name.append(None)
        try:
            dep_flight_aircraft_owner_airline_code.append(departure['flight']['owner']['code']['icao'])
        except (KeyError, TypeError):
            dep_flight_aircraft_owner_airline_code.append(None)


        #flight arrivals/departures details
        try:
            arr_came_from_country.append(arrival['flight']['airport']['origin']['position']['country']['name'])
        except (KeyError, TypeError):
            arr_came_from_country.append(None)
        try:
            arr_came_from_airport_name.append(arrival['flight']['airport']['origin']['name'])
        except (KeyError, TypeError):
            arr_came_from_airport_name.append(None)
        try:
            arr_came_from_airport_code.append(arrival['flight']['airport']['origin']['code']['icao'])
        except (KeyError, TypeError):
            arr_came_from_airport_code.append(None)
        try:
            arr_scheduled_departure.append(arrival['flight']['time']['scheduled']['departure'])
        except (KeyError, TypeError):
            arr_scheduled_departure.append(None)
        try:
            arr_scheduled_arrival.append(arrival['flight']['time']['scheduled']['arrival'])
        except (KeyError, TypeError):
            arr_scheduled_arrival.append(None)
        try:
            arr_real_departure.append(arrival['flight']['time']['real']['departure'])
        except (KeyError, TypeError):
            arr_real_departure.append(None)
        try:
            arr_real_arrival.append(arrival['flight']['time']['real']['arrival'])
        except (KeyError, TypeError):
            arr_real_arrival.append(None)
        try:
            arr_estimated_departure.append(arrival['flight']['time']['estimated']['departure'])
        except (KeyError, TypeError):
            arr_estimated_departure.append(None)
        try:
            arr_estimated_arrival.append(arrival['flight']['time']['estimated']['arrival'])
        except (KeyError, TypeError):
            arr_estimated_arrival.append(None)
        try:
            arr_duree.append(arrival['flight']['time']['other']['duration'])
        except (KeyError, TypeError):
            arr_duree.append(None)

        try:
            dep_country_destination.append(departure['flight']['airport']['destination']['position']['country']['name'])
        except (KeyError, TypeError):
            dep_country_destination.append(None)
        try:
            dep_airport_name_destination.append(departure['flight']['airport']['destination']['name'])
        except (KeyError, TypeError):
            dep_airport_name_destination.append(None)
        try:
            dep_airport_code_icao_destination.append(departure['flight']['airport']['destination']['code']['icao'])
        except (KeyError, TypeError):
            dep_airport_code_icao_destination.append(None)
        try:
            dep_scheduled_departure.append(departure['flight']['time']['scheduled']['departure'])
        except (KeyError, TypeError):
            dep_scheduled_departure.append(None)
        try:
            dep_scheduled_arrival.append(departure['flight']['time']['scheduled']['arrival'])
        except (KeyError, TypeError):
            dep_scheduled_arrival.append(None)
        try:
            dep_real_departure.append(departure['flight']['time']['real']['departure'])
        except (KeyError, TypeError):
            dep_real_departure.append(None)
        try:
            dep_real_arrival.append(departure['flight']['time']['real']['arrival'])
        except (KeyError, TypeError):
            dep_real_arrival.append(None)
        try:
            dep_estimated_departure.append(departure['flight']['time']['estimated']['departure'])
        except (KeyError, TypeError):
            dep_estimated_departure.append(None)
        try:
            dep_estimated_arrival.append(departure['flight']['time']['estimated']['arrival'])
        except (KeyError, TypeError):
            dep_estimated_arrival.append(None)
        try:
            dep_duree.append(departure['flight']['time']['other']['duration'])
        except (KeyError, TypeError):
            dep_duree.append(None)
    if 'weather' in airport_list[i]['airport']['pluginData']:
        for _ in range(occurences):
            try:
                sky_condition.append(airport_list[i]['airport']['pluginData']['weather']["sky"]["condition"]["text"])
            except (KeyError, TypeError):
                sky_condition.append(None)
            try:
                sky_visibility.append(str("{:.2f}".format(int(airport_list[i]['airport']['pluginData']['weather']["sky"]["visibility"]["nmi"])*1.852)))
            except (KeyError, TypeError):
                sky_visibility.append(None)
            try:
                wind_direction_degree = airport_list[i]['airport']['pluginData']['weather']["wind"]["direction"]["degree"]
                wind_direction_text = airport_list[i]['airport']['pluginData']['weather']["wind"]["direction"]["text"]
                wind_direction.append(f'{wind_direction_degree} {wind_direction_text}')
            except (KeyError, TypeError):
                wind_direction.append(None)
            try:
                wind_speed_desc.append(airport_list[i]['airport']['pluginData']['weather']["wind"]["speed"]["text"])
            except (KeyError, TypeError):
                wind_speed_desc.append(None)
            try:
                wind_speed_km.append(airport_list[i]['airport']['pluginData']['weather']["wind"]["speed"]["kmh"])
            except (KeyError, TypeError):
                wind_speed_km.append(None)
            try:
                wind_speed_kts.append(airport_list[i]['airport']['pluginData']['weather']["wind"]["speed"]["kts"])
            except (KeyError, TypeError):
                wind_speed_kts.append(None)
            try:
                temperature.append(airport_list[i]['airport']['pluginData']['weather']["temp"]["celsius"])
            except (KeyError, TypeError):
                temperature.append(None)
                
    else:
        for _ in range(occurences):
            sky_condition.append(None)
            sky_visibility.append(None)
            wind_direction.append(None)
            wind_speed_desc.append(None)
            wind_speed_km.append(None)
            wind_speed_kts.append(None)
            temperature.append(None)

In [113]:
print(len(dep_estimated_arrival))
print(len(sky_condition))

77805
77805


In [114]:
data_arrivals = []
data_departures = []
for c, a, arr_id, arr_stat, arr_plane_code, arr_plane_name,arr_plane_reg, arr_plane_owner_c, arr_plane_owner_airline_name, arr_plane_owner_airline_code, dep_id, dep_stat, dep_plane_code, dep_plane_name,dep_plane_reg, dep_plane_owner_c, dep_plane_owner_airline_name, dep_plane_owner_airline_code, arr_from, arr_name, arr_code, arr_sd, arr_sa, arr_rd, arr_ra, arr_ed, arr_ea, arr_d,dep_dest, dep_name, dep_code, dep_sd,dep_sa, dep_rd, dep_ra,dep_ed, dep_ea, dep_d,sc,sv,wd,wsd,wskm,wskt,temp in zip(country_list,all_airports,arr_flight_identification,arr_flight_status,arr_flight_aircraft_code,arr_flight_aircraft_name,arr_flight_aircraft_registration,arr_flight_aircraft_owner_country,arr_flight_aircraft_owner_airline_name,arr_flight_aircraft_owner_airline_code,dep_flight_identification,dep_flight_status,dep_flight_aircraft_code,dep_flight_aircraft_name,dep_flight_aircraft_registration,dep_flight_aircraft_owner_country,dep_flight_aircraft_owner_airline_name,dep_flight_aircraft_owner_airline_code,arr_came_from_country,arr_came_from_airport_name,arr_came_from_airport_code,arr_scheduled_departure,arr_scheduled_arrival,arr_real_departure,arr_real_arrival,arr_estimated_departure,arr_estimated_arrival,arr_duree,dep_country_destination,dep_airport_name_destination,dep_airport_code_icao_destination,dep_scheduled_departure,dep_scheduled_arrival,dep_real_departure,dep_real_arrival,dep_estimated_departure,dep_estimated_arrival,dep_duree,sky_condition,sky_visibility,wind_direction,wind_speed_desc,wind_speed_km,wind_speed_kts,temperature):
    if arr_sd is not None:
        arr_sd_formatted = datetime.utcfromtimestamp(arr_sd).strftime('%Y-%m-%d %H:%M:%S')
    else:
        arr_sd_formatted = None
    if arr_sa is not None:
        arr_sa_formatted = datetime.utcfromtimestamp(arr_sa).strftime('%Y-%m-%d %H:%M:%S')
    else:
        arr_sa_formatted = None
    if arr_rd is not None:
        arr_rd_formatted = datetime.utcfromtimestamp(arr_rd).strftime('%Y-%m-%d %H:%M:%S')
    else:
        arr_rd_formatted = None
    if arr_ra is not None:
        arr_ra_formatted = datetime.utcfromtimestamp(arr_ra).strftime('%Y-%m-%d %H:%M:%S')
    else:
        arr_ra_formatted = None
    if arr_ed is not None:
        arr_ed_formatted = datetime.utcfromtimestamp(arr_ed).strftime('%Y-%m-%d %H:%M:%S')
    else:
        arr_ed_formatted = None
    if arr_ea is not None:
        arr_ea_formatted = datetime.utcfromtimestamp(arr_ea).strftime('%Y-%m-%d %H:%M:%S')
    else:
        arr_ea_formatted = None
    if arr_d is not None:
        arr_d_formatted = datetime.utcfromtimestamp(arr_d).strftime('%Y-%m-%d %H:%M:%S')
    else:
        arr_d_formatted = None
    if dep_sd is not None:
        dep_sd_formatted = datetime.utcfromtimestamp(dep_sd).strftime('%Y-%m-%d %H:%M:%S')
    else:
        dep_sd_formatted = None
    if dep_sa is not None:
        dep_sa_formatted = datetime.utcfromtimestamp(dep_sa).strftime('%Y-%m-%d %H:%M:%S')
    else:
        dep_sa_formatted = None
    if dep_rd is not None:
        dep_rd_formatted = datetime.utcfromtimestamp(dep_rd).strftime('%Y-%m-%d %H:%M:%S')
    else:
        dep_rd_formatted = None
    if dep_ra is not None:
        dep_ra_formatted = datetime.utcfromtimestamp(dep_ra).strftime('%Y-%m-%d %H:%M:%S')
    else:
        dep_ra_formatted = None
    if dep_ed is not None:
        dep_ed_formatted = datetime.utcfromtimestamp(dep_ed).strftime('%Y-%m-%d %H:%M:%S')
    else:
        dep_ed_formatted = None
    if dep_ea is not None:
        dep_ea_formatted = datetime.utcfromtimestamp(dep_ea).strftime('%Y-%m-%d %H:%M:%S') 
    else:
        dep_ea_formatted = None
    if dep_d is not None:
        dep_d_formatted = datetime.utcfromtimestamp(dep_d).strftime('%Y-%m-%d %H:%M:%S')
    else:
        dep_d_formatted = None
    data_arrivals.append([c, a, arr_id, arr_stat, arr_plane_code, arr_plane_name,arr_plane_reg, arr_plane_owner_c, arr_plane_owner_airline_name, arr_plane_owner_airline_code, arr_from, arr_name, arr_code, arr_sd_formatted, arr_sa_formatted, arr_rd_formatted, arr_ra_formatted, arr_ed_formatted,arr_ea_formatted, arr_d_formatted,sc,sv,wd,wsd,wskm,wskt,temp])
    data_departures.append([c,a,dep_id, dep_stat, dep_plane_code, dep_plane_name,dep_plane_reg, dep_plane_owner_c, dep_plane_owner_airline_name, dep_plane_owner_airline_code,dep_dest, dep_name, dep_code, dep_sd_formatted,dep_sa_formatted, dep_rd_formatted, dep_ra_formatted,dep_ed_formatted,dep_ea_formatted, dep_d_formatted,sc,sv,wd,wsd,wskm,wskt,temp])

In [115]:
df_arrivals = pd.DataFrame(data_arrivals, columns=["Country","Airport", "Flight identification", "Flight Status", "Aircraft Code", "Aircraft Name", "Aircraft Registration","Aircraft owner (Country)","Airline Name","Airline Code", "Departure Country","Departure Airport Name","Departure Airport Code","Scheduled Departure","Scheduled Arrival","Real Departure","Real Arrival","Estimated Departure","Estimated Arrival","Flight Duration","Sky Condition","Sky Visibility","Wind Direction","Wind Speed Description","Wind Speed KM","Wind Speed Knots","Temperature"])
df_departures = pd.DataFrame(data_departures, columns=["Country","Airport", "Flight identification", "Flight Status", "Aircraft Code", "Aircraft Name", "Aircraft Registration","Aircraft owner (Country)","Airline Name","Airline Code", "Destination Country","Destination Airport Name","Destination Airport Code","Scheduled Departure","Scheduled Arrival","Real Departure","Real Arrival","Estimated Departure","Estimated Arrival","Flight Duration","Sky Condition","Sky Visibility","Wind Direction","Wind Speed Description","Wind Speed KM","Wind Speed Knots","Temperature"])

In [116]:
df_arrivals_sorted = df_arrivals.sort_values(by=['Country', 'Airport']).reset_index(drop=True)
df_departures_sorted = df_departures.sort_values(by=['Country', 'Airport']).reset_index(drop=True)

In [117]:
df_arrivals_sorted.head(10)

,Country,Airport,Flight identification,Flight Status,Aircraft Code,Aircraft Name,Aircraft Registration,Aircraft owner (Country),Airline Name,Airline Code,...,Estimated Departure,Estimated Arrival,Flight Duration,Sky Condition,Sky Visibility,Wind Direction,Wind Speed Description,Wind Speed KM,Wind Speed Knots,Temperature
0,Afghanistan,Herat International Airport,RQ101,Scheduled,737,,,None,None,None,...,None,None,None,None,None,None,None,NaN,NaN,NaN
1,Afghanistan,Herat International Airport,RQ147,Scheduled,737,,,None,None,None,...,None,None,None,None,None,None,None,NaN,NaN,NaN
2,Afghanistan,Herat International Airport,FG251,Scheduled,737,,,None,None,None,...,None,None,None,None,None,None,None,NaN,NaN,NaN
3,Afghanistan,Herat International Airport,RQ101,Scheduled,737,,SE-MFY,None,None,None,...,None,None,None,None,None,None,None,NaN,NaN,NaN
4,Afghanistan,Herat International Airport,FG251,Scheduled,737,Boeing 737 MAX 8,,None,None,None,...,None,None,None,None,None,None,None,NaN,NaN,NaN
5,Afghanistan,Kabul International Airport,RQ904,Estimated 05:21,B733,,,Afghanistan,Kam Air,KMF,...,None,2024-03-15 00:51:12,None,None,None,None,None,NaN,NaN,NaN
6,Afghanistan,Kabul International Airport,FG416,Estimated 06:35,B735,,,Afghanistan,Ariana Afghan Airlines,AFG,...,None,2024-03-15 02:05:52,None,None,None,None,None,NaN,NaN,NaN
7,Afghanistan,Kabul International Airport,FG402,Scheduled,734,,,None,None,None,...,2024-03-15 01:00:00,None,None,None,None,None,None,NaN,NaN,NaN
8,Afghanistan,Kabul International Airport,RQ994,Estimated 06:40,A343,,,Afghanistan,Kam Air,KMF,...,None,2024-03-15 02:10:08,None,None,None,None,None,NaN,NaN,NaN
9,Afghanistan,Kabul International Airport,RQ902,Scheduled,A343,,,Afghanistan,Kam Air,KMF,...,2024-03-15 00:00:00,None,None,None,None,None,None,NaN,NaN,NaN


In [118]:
df_departures_sorted.head(10)

,Country,Airport,Flight identification,Flight Status,Aircraft Code,Aircraft Name,Aircraft Registration,Aircraft owner (Country),Airline Name,Airline Code,...,Estimated Departure,Estimated Arrival,Flight Duration,Sky Condition,Sky Visibility,Wind Direction,Wind Speed Description,Wind Speed KM,Wind Speed Knots,Temperature
0,Afghanistan,Herat International Airport,RQ102,Scheduled,737,,,None,None,None,...,None,None,None,None,None,None,None,NaN,NaN,NaN
1,Afghanistan,Herat International Airport,RQ148,Scheduled,737,,,None,None,None,...,None,None,None,None,None,None,None,NaN,NaN,NaN
2,Afghanistan,Herat International Airport,FG252,Scheduled,737,,,None,None,None,...,None,None,None,None,None,None,None,NaN,NaN,NaN
3,Afghanistan,Herat International Airport,RQ102,Scheduled,737,,,None,None,None,...,None,None,None,None,None,None,None,NaN,NaN,NaN
4,Afghanistan,Herat International Airport,FG252,Scheduled,737,,,None,None,None,...,None,None,None,None,None,None,None,NaN,NaN,NaN
5,Afghanistan,Kabul International Airport,RQ923,Scheduled,737,Airbus A321-272NX,N506AS,None,None,None,...,None,None,None,None,None,None,None,NaN,NaN,NaN
6,Afghanistan,Kabul International Airport,FG703,Scheduled,310,,N564AS,None,None,None,...,None,None,None,None,None,None,None,NaN,NaN,NaN
7,Afghanistan,Kabul International Airport,G9719,Scheduled,320,,N627AS,None,None,None,...,None,None,None,None,None,None,None,NaN,NaN,NaN
8,Afghanistan,Kabul International Airport,FZ302,Estimated dep 09:30,73H,,,None,None,None,...,2024-03-15 05:00:00,None,None,None,None,None,None,NaN,NaN,NaN
9,Afghanistan,Kabul International Airport,FG267,Scheduled,737,,N529AS,None,None,None,...,None,None,None,None,None,None,None,NaN,NaN,NaN


In [119]:
df_arrivals_sorted.to_csv(f'./data/df_arrivals_final.csv',index=False)
df_departures_sorted.to_csv(f'./data/df_departures_final.csv',index=False)